<a href="https://colab.research.google.com/github/wideowsieci/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [12]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_clases = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train,
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks= [tensorboard_callback]
    )

    return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_clases):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),  
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),  
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),  
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),  
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_clases, activation='softmax'),

])

In [10]:
model = get_cnn_v5(input_shape, num_clases)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.8631 - accuracy: 0.2209 - val_loss: 1.2210 - val_accuracy: 0.6215
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 1.0742 - accuracy: 0.6589 - val_loss: 0.3505 - val_accuracy: 0.8838
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.5130 - accuracy: 0.8368 - val_loss: 0.1190 - val_accuracy: 0.9644
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.3174 - accuracy: 0.9036 - val_loss: 0.0918 - val_accuracy: 0.9727
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2409 - accuracy: 0.9285 - val_loss: 0.0535 - val_accuracy: 0.9849


0.9664399092970521

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),  
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),  
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),
  
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),  
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),  
  Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),  
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_clases, activation='softmax'),

])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
        X_train,
        y_train,
        batch_size=int(params.get('batch_size', 128)),
        epochs=5,
        verbose=0,
     )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

             

In [28]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4004294229582801, 'dropout_cnn_block_three': 0.4365089723911838, 'dropout_cnn_block_two': 0.3043583321983475, 'dropout_dense_block_one': 0.5092283047254271, 'dropout_dense_block_two': 0.32248346990322674}
accuracy=0.9603174328804016
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.37318504662913377, 'dropout_cnn_block_three': 0.4823372796516694, 'dropout_cnn_block_two': 0.3971714244298602, 'dropout_dense_block_one': 0.5301343400337378, 'dropout_dense_block_two': 0.605308885960028}
accuracy=0.902721107006073
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4202127319010638, 'dropout_cnn_block_three': 0.38141327436087535, 'dropout_cnn_block_two': 0.34961447970302684, 'dropout_dense_block_one': 0.600547537722658, 'dropout_dense_block_two': 0.5824890834784138}
accuracy=0.8882086277008057
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.42895681832710686, 'dropout_cnn_block_three': 0.3992432803517023, 'dropout_cnn_block_two': 0.43808179692719